In [1]:
from PIL import Image, ImageColor, ImageFont, ImageDraw
import matplotlib as plt
import numpy as np
import os
import pandas as pd
import random

In [2]:
# get the list of files to work with
lof = list()
directory = './src/'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        lof.append(filename)

In [3]:
# create 9 small random images
# 1 divide the image into 9 sections:

# [row, column]:

sections = [[1, 1], [1, 2], [1, 3],
            [2, 1], [2, 2], [2, 3],
            [3, 1], [3, 2], [3, 3]]

# and generate (and name) the small images accordingly

In [10]:
# to resize to XXX w or XXX L, whatever is bigger - this will be im_0.png

default_size = 900

out_dir = './img/'

In [11]:
df_lof = pd.DataFrame(lof, columns = ['file'])
df_lof['filename'] = df_lof['file'].str.split('.').str[0]
df_lof['extension'] = df_lof['file'].str.split('.').str[1]

In [12]:
df_lof['extension'].value_counts()

extension
jpg     47
webp     1
Jpg      1
Name: count, dtype: int64

In [13]:
# check if you really need all these images, define the dataframe accordingly. we'll continue from here

needed = df_lof

In [14]:
def r_w_h(i):
    # read w and h of one file, get the bigger value

    the_file = f'{directory}{needed['file'][i]}'
    #filename:
    fn = needed['filename'][i]
    
    # read the image
    im = Image.open(the_file)
    width, height = im.size
    # get values sorted, bigger will be selected later
    s_sort = list(im.size)
    s_sort.sort()
    #print(f'im: {im}, w, h: {width}, {height}, s: {s_sort}, fn: {fn}')
    return im, width, height, s_sort, fn



def resize_image(i):
    im, width, height, s_sort, fn = r_w_h(i)
    if default_size == s_sort[1]:
        new_file_name = f'{out_dir}{fn}.png'
        resized = im
        resized.save(new_file_name, format('png'))
        #print(f'Not resized, size: {resized.size}')
        return resized, fn
        
    else:
        if width > height:
            new_h = int((default_size/width*height))
            resized = im.resize((default_size, new_h), Image.Resampling.NEAREST)
        elif width < height:
            new_w = int((default_size/height*width))
            resized = im.resize((new_w, default_size), Image.Resampling.NEAREST)
        else:
            resized = im.resize((default_size, default_size), Image.Resampling.NEAREST)
        new_file_name = f'{out_dir}{fn}.png'
        resized.save(new_file_name, format('png'))
        #print(f'Resized, size: {resized.size}')
        return resized, fn
        
        #resized.show()


def generate_fractions(i):
    print(i)
    resized, fn = resize_image(i)
    
    # calculate rectangle for section
    c_left = 0
    c_up = 0
    c_w = resized.size[0]//3
    c_h = resized.size[1]//3
    for e, rc in enumerate(sections):
        # define crop location
        c_left += c_w*(rc[1]-1)
        c_up += c_h*(rc[0]-1)
        c_right = c_left + c_w
        c_low = c_up + c_h
        cl = (c_left, c_up, c_right, c_low)
        #print(e+1, rc, cl)
        c_left = 0
        c_up = 0
    
        # get a random small square part of the section
    
        part = random.randrange(2, 3)
        
        # define crop size
        
        if c_w < c_h:
            im_w = int(c_w/part)
            im_h = im_w
        else:
            im_h = int(c_h/part)
            im_w = im_h
        #print(f'{im_w}, {im_h}', part)
        
        #image = Image.new(mode = "RGB", size = (im_w, im_h), color = li_col)
    
        
        # define crop location
        left = random.randrange(c_left, c_right - im_w)
        upper = random.randrange(c_up, c_low - im_h)
        right = left + im_w
        lower = upper + im_h
        
        # show cropped part 
        
        nim = resized.crop((left, upper, right, lower))
        nim = nim.resize((default_size//2, default_size//2), Image.Resampling.NEAREST)
        new_file_name = f'{out_dir}{fn}_{e+1}.png'
        nim.save(new_file_name, format('png'))
        #nim.show()

In [15]:
for i in range(len(needed)):
    #generate_fractions(i)
    resize_image(i)

In [ ]:
# add resized images to the page, which shows the selected image

In [ ]:
# generate QR codes for the images

In [11]:
# generate QR codes (card info, questionms text, link to image(s)

qr_dir = './qrs/'

In [9]:
# get a random small square part of the file

part = random.randrange(4, 6)

# define crop size

if width < height:
    im_w = int((width - (width/part))/part)
    im_h = im_w
else:
    im_h = int((height - (height/part))/part)
    im_w = im_h
print(f'{im_w}, {im_h}')

#image = Image.new(mode = "RGB", size = (im_w, im_h), color = li_col)

163, 163


In [23]:
# define crop location
left = random.randrange(0, width - im_w)
upper = random.randrange(0, height - im_h)
right = left + im_w
lower = upper + im_h

# show cropped part 

nim = im.crop((left, upper, right, lower))
nim.save('ims/c3.jpg')
#nim.show()


In [11]:
# generate a qr code for the solution
df = pd.read_csv('/home/balibaa/Downloads/prio/gwa/HU/hu_cardlist.csv', sep = ';', names = ['ID', 'artist', 'art'])
df.head()

,ID,artist,art
0,1,Kolozsvári Márton és György,Szent György bronzszobra (Prága)
1,2,Id. Markó Károly,Champagnai táj
2,3,Ferenczy István,Kölcsey Ferenc szobra
3,4,Brocky Károly,Pihenő Pszyché
4,5,Barabás Miklós,Önarckép


In [12]:
import datetime
from time import sleep
import qrcode
from PIL import Image

def ts_uid():
    '''Returns a HEX UID based on the current timestamp.'''

    time_stamp = datetime.datetime.utcnow().strftime('%Y%m%d%H%M%S%f')
    time_stamp_hex = hex(int(time_stamp))
    ts_uid = time_stamp_hex[2:]
    # Sleep 0.1 millisecond to make sure all UIDs are really unique.
    # This is important if UIDs are created in batches with loops.
    sleep(0.0001)
    return ts_uid

def input_uid(idint):
    '''Returns a HEX UID based on input.'''

    time_stamp = datetime.datetime.utcnow().strftime('%Y%m%d%H%M%S%f')
    time_stamp_hex = hex(int(time_stamp))
    ts_uid = time_stamp_hex[2:]
    # Sleep 0.1 millisecond to make sure all UIDs are really unique.
    # This is important if UIDs are created in batches with loops.
    sleep(0.0001)
    return ts_uid

def make_uid(inp):
    '''Returns a HEX UID based on input.'''
    if inp == 'ts':
        input_to_convert = datetime.datetime.utcnow().strftime('%Y%m%d%H%M%S%f')
        input_to_hex = hex(int(input_to_convert))
        ts_uid = input_to_hex[2:]
        # Sleep 0.1 millisecond to make sure all UIDs are really unique.
        # This is important if UIDs are created besed on timestamp in batches with loops.
        sleep(0.0001)
    elif type(inp) == int:
        input_to_convert = (inp)
        input_to_hex = hex(int(input_to_convert))
        ts_uid = input_to_hex[2:]
    elif type(inp) == str:
        input_to_convert = inp.encode('utf-8')
        input_to_hex = input_to_convert.hex()
        ts_uid = input_to_hex[2:]
    else:
        print('Expecting "ts", int or string input.')
        return None
    return ts_uid

In [13]:
qr_data = str(f'{df['ID'][32]}. {df['artist'][32]}: {df['art'][32]}')
qrcode.make(qr_data).show()
print(qr_data)

33. Csontváry Kosztka Tivadar: Magános cédrus


In [14]:
# pixel conversion rate: 1mm = 3.77 pixel

c_h = 320
c_w = 207

draw = ImageDraw.Draw(im)
text = f'{df['ID'][32]}'
font = ImageFont.truetype('NotoSansMath-Regular.ttf', 45)
#textwidth, textheight = draw.textsize(text, font)

# calculate the x,y coordinates of 
# the text
x = width - 150
y = height -150

# draw watermark in the bottom right
# corner
draw.text((x, y), text, font=font, fill = (0, 0, 0))
im.show()